# Project Euler
## [Problem 31](https://projecteuler.net/problem=31)
### Coin sums

<p>In the United Kingdom the currency is made up of pound (£) and pence (p). There are eight coins in general circulation:</p>
<blockquote>1p, 2p, 5p, 10p, 20p, 50p, £1 (100p), and £2 (200p).</blockquote>
<p>It is possible to make £2 in the following way:</p>
<blockquote>1×£1 + 1×50p + 2×20p + 1×5p + 1×2p + 3×1p</blockquote>
<p>How many different ways can £2 be made using any number of coins?</p>

### Solution
I found a YouTube video explaining how to solve the general problem here:

> [Coin Change Problem Number of ways to get total | Dynamic Programming | Algorithms](https://www.youtube.com/watch?v=L27_JpN6Z1Q)

I then applied it to this problem.

In [1]:
UK_COINS = (1, 2, 5, 10, 20, 50, 100, 200)

In [2]:
def coin_combo_array(coins: list, total: int):
    """
    Create an array of the number of possible ways to represent
    a value with coin denominations for all values from 0 to total.
    """
    coin_combos =  [[0 for coin in coins] for i in range(total+1)]
    for amount in range(total+1):
        for i, coin_value in enumerate(coins):
            if amount == 0:
                coin_combos[amount][i] = 1
            elif coin_value == 1:
                coin_combos[amount][i] = 1
            elif i == 0:
                if (amount % coin_value) == 0:
                    coin_combos[amount][i] = 1
                else:
                    coin_combos[amount][i] = 0                    
            elif total < coin_value:
                coin_combos[amount][i] = coin_combos[amount][i-1]
            else:
                coin_combos[amount][i] = coin_combos[amount][i-1] + coin_combos[amount-coin_value][i]
    
    return coin_combos


def coin_combinations(coins: list, total: int):
    """
    Return the number of ways to represent total with numeric denominations
    given by coins.
    """
    return coin_combo_array(coins, total)[-1][-1]

In [3]:
coin_combinations(UK_COINS, 200)

73682

For fun, I ran this problem with US currency and found the combinations of ways to make \$1.00.

In [4]:
US_COINS = (1, 5, 10, 25, 50)
combos = coin_combinations(US_COINS, 100)
print(combos)

292


Below, I tested the time it took to compute a large dollar amount to see how
efficient the algorithm was. Try different values for the cell below to 
test both the speed of your computer and the speed of the algorithm.

In [5]:
from time import time

US_CURRENCY = (1, 5, 10, 25, 50, 100, 200, 500, 1000, 2000, 5000, 10000)
LARGE_TOTAL = 1000000  # $10,000.00

start_time = time()
large_dollar_combos = coin_combinations(US_CURRENCY, LARGE_TOTAL)
end_time = time()

In [6]:
print(f"Found the {large_dollar_combos:,} combinations of ${LARGE_TOTAL//100:,.2f} "
      f"with US currency in {(end_time-start_time):.3f} seconds.")

Found the 444,253,458,330,064,869,987,757,337,916,426 combinations of $10,000.00 with US currency in 7.112 seconds.


After reading the PDF for the solution, I found out that you can cut the
number of calculations and shrink the array to one dimension by only
iterating from the current coin value onward. This cut the processing time by
1/2 for the large dollar amount. The revised functions are below.

In [7]:
def coin_combo_array_fast(coins: list, total: int) -> list:
    coin_combos = [0 for i in range(total+1)]
    coin_combos[0] = 1
    for coin_value in coins:
        for amount in range(coin_value, total+1):
            if coin_value == 1:
                coin_combos[amount] = 1
            else:
                coin_combos[amount] += coin_combos[amount-coin_value]
    
    return coin_combos


def coin_combinations_fast(coins, total) -> int:
    """
    Return the number of ways to represent total with numeric denominations
    given by coins.
    """
    return coin_combo_array_fast(coins, total)[-1]

In [8]:
start_time = time()
fast_combo = coin_combinations_fast(US_CURRENCY, LARGE_TOTAL)
end_time = time()

In [9]:
print(f"Found the {fast_combo:,} combinations of ${LARGE_TOTAL//100:,.2f} "
      f"with US currency in {(end_time-start_time):.3f} seconds.")

Found the 444,253,458,330,064,869,987,757,337,916,426 combinations of $10,000.00 with US currency in 2.441 seconds.


## Similar problem: Fewest coins

A problem with a similar solution is finding the way to represent an amount
of money with the fewest denominations of currency. The solution to that
problem is below.

In [10]:
def fewest_coins_array(coins: list, total: int):
    coin_amounts =  [0 for n in range(total+1)]
    coin_array = [0 for n in range(total+1)]
    for coin_value in coins:
        for amount in range(coin_value, total+1):
            if amount == coin_value:
                coin_amounts[amount] = 1
                coin_array[amount] = coin_value
            elif (amount % coin_value) == 0:
                coin_amounts[amount] = amount // coin_value
                coin_array[amount] = coin_value
            else:
                min_coins = min(coin_amounts[amount], 1 + coin_amounts[amount-coin_value])
                if min_coins < coin_amounts[amount]:
                    coin_amounts[amount] = min_coins
                    coin_array[amount] = coin_value
    
    return coin_amounts, coin_array


def number_of_fewest_coins(coins: list, total: int) -> int:
    return fewest_coins_array(coins, total)[0][-1]


def fewest_coin_values(coins: list, total: int) -> tuple:
    coin_amounts, coin_array = fewest_coins_array(coins, total)
    if coin_amounts[-1] == 0:
        return (0,)  # This means it is impossible to represent total with the given coins.
    total_in_fewest_coins = []
    amount_left = total
    while amount_left > 0:
        current_coin = coin_array[amount_left]
        while amount_left >= current_coin:
            total_in_fewest_coins.append(current_coin)
            amount_left -= current_coin
    
    total_in_fewest_coins.sort()
    
    return tuple(total_in_fewest_coins)

#### Example: 99 cents

In [11]:
start_time = time()
print("Fewest coins to make 99 cents:", number_of_fewest_coins(US_COINS, 99))
print("Coin values:", fewest_coin_values(US_COINS, 99))
end_time = time()
print("Time:", f"{end_time-start_time:.5g} seconds")

Fewest coins to make 99 cents: 8
Coin values: (1, 1, 1, 1, 10, 10, 25, 50)
Time: 0.00099802 seconds


### For fun: Least amount of coins with at least one of each coin.

In [12]:
def all_coins(coins: list, total: int) -> tuple:
    if not isinstance(coins, list):
        coins = list(coins)
    coins.sort()
    all_coins = [coin for coin in coins]
    all_coins_sum = sum(all_coins)
    while all_coins_sum > total:
        largest_coin = all_coins.pop()
        all_coins_sum -= largest_coin
    
    remaining = total - all_coins_sum
    while remaining > 0:
        for coin in reversed(coins):
            if coin <= remaining:
                for _ in range(remaining//coin):
                    all_coins.append(coin)
                    remaining -= coin
    
    all_coins.sort()
    
    return tuple(all_coins)

In [13]:
print(all_coins(US_COINS, 100))

(1, 1, 1, 1, 1, 5, 5, 10, 25, 50)
